In [1]:
import pandas as pd

data_path = '/kaggle/input/cat-in-the-dat/'

train = pd.read_csv(data_path+'train.csv', index_col='id')
test = pd.read_csv(data_path+'test.csv', index_col='id')
submission = pd.read_csv(data_path+'sample_submission.csv', index_col = 'id')

print(train.shape, test.shape)

display(train.head())

(300000, 24) (200000, 23)


,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,nom_4,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,T,Y,Green,Triangle,Snake,Finland,Bassoon,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0
1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,Piano,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0
2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,Theremin,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0
3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,Oboe,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1
4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,Oboe,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0


# EDA

In [2]:
import pandas as pd

def resumetable(df):
    print(f'데이터셋 형상: {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['데이터 타입'])
    summary = summary.reset_index()
    summary = summary.rename(columns={'index': '피처'})
    summary['결측값 개수'] = df.isnull().sum().values
    summary['고윳값 개수'] = df.nunique().values
    summary['첫 번째 값'] = df.loc[0].values
    summary['두 번째 값'] = df.loc[1].values
    summary['세 번째 값'] = df.loc[2].values
    
    return summary
resumetable(train)

데이터셋 형상: (300000, 24)


,피처,데이터 타입,결측값 개수,고윳값 개수,첫 번째 값,두 번째 값,세 번째 값
0,bin_0,int64,0,2,0,0,0
1,bin_1,int64,0,2,0,1,0
2,bin_2,int64,0,2,0,0,0
3,bin_3,object,0,2,T,T,F
4,bin_4,object,0,2,Y,Y,Y
5,nom_0,object,0,3,Green,Green,Blue
6,nom_1,object,0,6,Triangle,Trapezoid,Trapezoid
7,nom_2,object,0,6,Snake,Hamster,Lion
8,nom_3,object,0,6,Finland,Russia,Russia
9,nom_4,object,0,4,Bassoon,Piano,Theremin


In [3]:
train['bin_3'] = train['bin_3'].map({'F':0, 'T':1})
train['bin_4'] = train['bin_4'].map({'N':0, 'Y':1})

ord1dict = {'Novice':0, 'Contributor':1, 'Expert':2, 'Master':3, 'Grandmaster':4}
train['ord_1'] = train['ord_1'].map(ord1dict)

ord2dict = {'Freezing':0, 'Cold':1, 'Warm':2, 'Hot':3, 'Boiling Hot':4, 'Lava Hot':5}
train['ord_2'] = train['ord_2'].map(ord2dict)

In [4]:
# from sklearn.preprocessing import OrdinalEncoder

# oe = OrdinalEncoder(categories='auto')
# train[ordvar[3:]] = oe.fit_transform(train[ordvar[3:]])
# for var, cl in zip(ordvar[3:], oe.categories_):
#     print(var)
#     print(cl)

In [5]:
# import pandas as pd

# def resumetable(df):
#     print(f'데이터셋 형상: {df.shape}')
#     summary = pd.DataFrame(df.dtypes, columns=['데이터 타입'])
#     summary = summary.reset_index()
#     summary = summary.rename(columns={'index': '피처'})
#     summary['결측값 개수'] = df.isnull().sum().values
#     summary['고윳값 개수'] = df.nunique().values
#     summary['첫 번째 값'] = df.loc[0].values
#     summary['두 번째 값'] = df.loc[1].values
#     summary['세 번째 값'] = df.loc[2].values
    
#     return summary
# resumetable(train)

# Train, Test 합치기

In [6]:
import pandas as pd

data_path = '/kaggle/input/cat-in-the-dat/'

train = pd.read_csv(data_path+'train.csv', index_col='id')
test = pd.read_csv(data_path+'test.csv', index_col='id')
submission = pd.read_csv(data_path+'sample_submission.csv', index_col='id')

In [7]:
all_data = pd.concat([train, test], axis=0)
all_data = all_data.drop(['target'], axis=1)
all_data.shape

(500000, 23)

# OneHotEncoder

In [8]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
all_data_encoded = encoder.fit_transform(all_data)
all_data_encoded.shape

(500000, 16552)

In [9]:
from sklearn.model_selection import train_test_split

num_train = len(train)
X_train = all_data_encoded[:num_train]
display(X_train)
X_test = all_data_encoded[num_train:]
y = train['target']

print(X_train.shape)
print(y.shape)

X_train, X_val, y_train, y_val = train_test_split(X_train, y, test_size=0.1, stratify=y, random_state=10)

<300000x16552 sparse matrix of type '<class 'numpy.float64'>'
	with 6900000 stored elements in Compressed Sparse Row format>

(300000, 16552)
(300000,)


In [27]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

def y_pred(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
#      print(score = accuracy_score(y_pred, y_val))
    print(roc_auc_score(y_val, y_pred))
    return y_pred

print("xgb_model")
xgb_model = y_pred(XGBClassifier())
print("xgb_model_dart")
xgb_model_dart = y_pred(XGBClassifier(booster='dart', one_drop=True))
print("RF_model")
RF_model = y_pred(RandomForestClassifier(random_state=2))
print("LOR")
LOR = y_pred(LogisticRegression(max_iter=10000))
print("xgb_model_tyuning")
xgb_model_tyuning = y_pred(XGBClassifier(max_depth=2, n_estimators=500, learning_rate=0.1))

# xgb_model = XGBClassifier(max_depth=2, n_estimators=500, learning_rate=0.1)

xgb_model
0.6261970000746908
xgb_model_dart
0.6127524625365207
RF_model
0.5267188226916767
LOR
0.6844410044610185
xgb_model_tyuning
0.6018132705841057


In [28]:
import numpy as np
df_pred = pd.DataFrame(data= np.c_[xgb_model, xgb_model_dart, 
                                   RF_model, LOR, xgb_model_tyuning],
                       columns=['xgb_model', 'xgb_model_dart', 'RF_model', 'LOR', 'xgb_model_tyuning'])

df_pred.corr()

,xgb_model,xgb_model_dart,RF_model,LOR,xgb_model_tyuning
xgb_model,1.000000,0.763892,0.355628,0.538895,0.751764
xgb_model_dart,0.763892,1.000000,0.382718,0.496402,0.814154
RF_model,0.355628,0.382718,1.000000,0.257837,0.425361
LOR,0.538895,0.496402,0.257837,1.000000,0.487662
xgb_model_tyuning,0.751764,0.814154,0.425361,0.487662,1.000000


Best Score : LOR

0. (1 case) LOR
1. (2 caes) LOR + 상관관계가 가장 낮은 RF
2. (2 caes) LOR + 상관관계가 가장 높은 xgb_model
3. (3 caes) LOR + 상관관계 Min + 상관관계 Max
4. (5 caes) All

In [12]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

print("xgb_model")
xgb_model = XGBClassifier().fit(X_train, y_train)

print("xgb_model_dart")
xgb_model_dart = XGBClassifier(booster='dart', one_drop=True).fit(X_train, y_train)

print("RF_model")
RF_model = RandomForestClassifier(random_state=2).fit(X_train, y_train)

print("LOR")
LOR = LogisticRegression(max_iter=10000).fit(X_train, y_train)

print("xgb_model_tyuning")
xgb_model_tyuning = XGBClassifier(max_depth=2, n_estimators=500, learning_rate=0.1).fit(X_train, y_train)


xgb_model
xgb_model_dart
RF_model
LOR
xgb_model_tyuning


In [22]:
print("single_model_ver01 예측값 연산")
single_model_preds_ver00 = LOR.predict(X_test)
submission['count'] = single_model_preds_ver00
submission.to_csv('submission_ver00.csv', index=False) 
# submission

single_model_ver01 예측값 연산


In [26]:
single_model_preds_ver00 = LOR.predict(X_val)
score_LOR = roc_auc_score(y_val, single_model_preds_ver00)
print(score_LOR)

0.6844410044610185


In [18]:
from sklearn.ensemble import VotingClassifier
def single_model_ver01():
    single_models = [
    ('LOR', LOR),
    ('RF_model', RF_model)
    ]

    # voting regressor 만들기
    print("voting_classifier 모델 생성")
    voting_classifier = VotingClassifier(single_models,voting='soft', n_jobs=-1)
    voting_classifier.fit(X_train, y_train)
    print("y_pred 계산 시작")
    y_pred = voting_classifier.predict(X_val)
    print("스코어 계산")
    score = roc_auc_score(y_val, y_pred)
    print(score)
    return score, voting_classifier

print("single_model:             ",single_model_ver01())

# 모델 성능 검증
print("single_model_ver01 연산")
single_model_ver01 = single_model()[1]

print("single_model_ver01 예측값 연산")
single_model_preds_ver01 = single_model_ver01.predict(X_test)
submission['count'] = single_model_preds_ver01
submission.to_csv('submission_ver01.csv', index=False) 
# submission

voting_classifier 모델 생성
y_pred 계산 시작
스코어 계산
0.6263668756558475
single_model:              (0.6263668756558475, VotingClassifier(estimators=[('LOR', LogisticRegression(max_iter=10000)),
                             ('RF_model',
                              RandomForestClassifier(random_state=2))],
                 n_jobs=-1, voting='soft'))
single_model_ver01 연산
voting_classifier 모델 생성
y_pred 계산 시작
스코어 계산
0.6263668756558475
single_model_ver01 예측값 연산


In [19]:
from sklearn.ensemble import VotingClassifier
def single_model_ver02():
    single_models = [
    ('LOR', LOR),
    ('xgb_model', xgb_model)
    ]

    # voting regressor 만들기
    print("voting_classifier 모델 생성")
    voting_classifier = VotingClassifier(single_models,voting='soft', n_jobs=-1)
    voting_classifier.fit(X_train, y_train)
    print("y_pred 계산 시작")
    y_pred = voting_classifier.predict(X_val)
    print("스코어 계산")
    score = roc_auc_score(y_val, y_pred)
    print(score)
    return score, voting_classifier

print("single_model_ver02:             ",single_model_ver02())

# 모델 성능 검증
print("single_model_ver02 연산")
single_model_ver02 = single_model_ver02()[1]

print("single_model_ver02 예측값 연산")
single_model_preds_ver02 = single_model_ver02.predict(X_test)
submission['count'] = single_model_preds_ver02
submission.to_csv('submission_ver02.csv', index=False) 
# submission

voting_classifier 모델 생성
y_pred 계산 시작
스코어 계산
0.6637208308031676
single_model_ver02:              (0.6637208308031676, VotingClassifier(estimators=[('LOR', LogisticRegression(max_iter=10000)),
                             ('xgb_model',
                              XGBClassifier(base_score=0.5, booster='gbtree',
                                            callbacks=None, colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=1,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None, gamma=0,
                                            gpu_id=-1, grow_policy='depthwise',
                                            importance_type=None,
                                            interaction_constraints='',
                                            lear

In [20]:
from sklearn.ensemble import VotingClassifier
def single_model_ver03():
    single_models = [
    ('LOR', LOR),
    ('xgb_model', xgb_model),
    ('RF_model', RF_model)
    ]

    # voting regressor 만들기
    print("voting_classifier 모델 생성")
    voting_classifier = VotingClassifier(single_models,voting='soft', n_jobs=-1)
    voting_classifier.fit(X_train, y_train)
    print("y_pred 계산 시작")
    y_pred = voting_classifier.predict(X_val)
    print("스코어 계산")
    score = roc_auc_score(y_val, y_pred)
    print(score)
    return score, voting_classifier

print("single_model_ver03:             ",single_model_ver03())

# 모델 성능 검증
print("single_model_ver03 연산")
single_model_ver03 = single_model_ver03()[1]

print("single_model_ver03 예측값 연산")
single_model_preds_ver03 = single_model_ver03.predict(X_test)
submission['count'] = single_model_preds_ver03
submission.to_csv('submission_ver03.csv', index=False) 
# submission

voting_classifier 모델 생성
y_pred 계산 시작
스코어 계산
0.6252641392585377
single_model_ver03:              (0.6252641392585377, VotingClassifier(estimators=[('LOR', LogisticRegression(max_iter=10000)),
                             ('xgb_model',
                              XGBClassifier(base_score=0.5, booster='gbtree',
                                            callbacks=None, colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=1,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None, gamma=0,
                                            gpu_id=-1, grow_policy='depthwise',
                                            importance_type=None,
                                            interaction_co...nts='',
                                            learnin

In [21]:
from sklearn.ensemble import VotingClassifier
def single_model_ver04():
    single_models = [
    ('LOR', LOR),
    ('xgb_model', xgb_model),
    ('RF_model', RF_model),
    ('xgb_model_dart', xgb_model_dart),
    ('xgb_model_tyuning', xgb_model_tyuning)        
    ]

    # voting regressor 만들기
    print("voting_classifier 모델 생성")
    voting_classifier = VotingClassifier(single_models,voting='soft', n_jobs=-1)
    voting_classifier.fit(X_train, y_train)
    print("y_pred 계산 시작")
    y_pred = voting_classifier.predict(X_val)
    print("스코어 계산")
    score = roc_auc_score(y_val, y_pred)
    print(score)
    return score, voting_classifier

print("single_model:             ",single_model_ver04())

# 모델 성능 검증
print("single_model_ver04 연산")
single_model_ver04 = single_model_ver04()[1]

print("single_model_ver04 예측값 연산")
single_model_preds_ver04 = single_model_ver04.predict(X_test)
submission['count'] = single_model_preds_ver04
submission.to_csv('submission_ver04.csv', index=False) 
# submission


# print("xgb_model")
# xgb_model = XGBClassifier().fit(X_train, y_train)

# xgb_model_dart = XGBClassifier(booster='dart', one_drop=True).fit(X_train, y_train)

# print("RF_model")
# RF_model = RandomForestClassifier(random_state=2).fit(X_train, y_train)

# print("LOR")
# LOR = LogisticRegression(max_iter=10000).fit(X_train, y_train)



voting_classifier 모델 생성
y_pred 계산 시작
스코어 계산
0.6180547996225936
single_model:              (0.6180547996225936, VotingClassifier(estimators=[('LOR', LogisticRegression(max_iter=10000)),
                             ('xgb_model',
                              XGBClassifier(base_score=0.5, booster='gbtree',
                                            callbacks=None, colsample_bylevel=1,
                                            colsample_bynode=1,
                                            colsample_bytree=1,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None, gamma=0,
                                            gpu_id=-1, grow_policy='depthwise',
                                            importance_type=None,
                                            interaction_co...
                                            gpu_id=-1, grow_poli

In [ ]:
# from sklearn.ensemble import VotingClassifier
# from xgboost import XGBClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import StratifiedKFold, cross_val_score
# kfold = StratifiedKFold(n_splits=3)

# estimators_ver03 = []

# LOR = LogisticRegression(max_iter=10000)
# estimators_ver03.append(('logistic', LOR))
# print("1 번째 모델 학습 완료")

# xgb_model = XGBClassifier(max_depth=2, n_estimators=500, learning_rate=0.1)
# estimators_ver03.append(('XGB', xgb_model))
# print("2 번째 모델 학습 완료")

# RF_model = RandomForestClassifier(random_state=2)
# estimators_ver03.append(('RF', RF_model))
# print("3 번째 모델 학습 완료")

# ensemble_ver03 = VotingClassifier(estimators_ver03)
# scores_ver03 = cross_val_score(ensemble_ver03, X_train, y_train, cv=kfold)
# print('ver03의Voting 결과: %.3f' % scores_ver03.mean())

In [ ]:
# y_preds_ver03 = voting_ensemble_ver03.predict_proba(X_test)[:, 1]
# print(roc_auc_score(y_val, y_pred_ver03))

# submission['target'] = y_preds_ver03
# submission.to_csv('submission_ver03.csv')

In [ ]:
# voting_ensemble_ver01 = VotingClassifier(estimators=[("logistic_model", logistic_model), ("RF_model", RF_model)],
#                                   voting = 'soft')

# voting_ensemble_ver01.fit(X_train, y_train)
# # y_pred_ver01 = voting_ensemble_ver01.predict(X_test)


# y_preds_ver01 = voting_ensemble_ver01.predict_proba(X_test)[:, 1]
# print(roc_auc_score(y_val, y_pred_ver01))

# submission['target'] = y_preds_ver01
# submission.to_csv('submission_ver03.csv')